In [1]:
%matplotlib notebook

import itertools
import logging
from functools import partial
from collections import defaultdict

import gensim
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors as plt_colors
import numpy as np
import pandas as pnd
import os
from sklearn.cluster import *
from sklearn.preprocessing import normalize
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA, RandomizedPCA
from sklearn.manifold import TSNE
from sklearn import svm, metrics
from multiprocessing import Pool
from knub.thesis.util import *

pnd.set_option("display.max_colwidth", 200)

/opt/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
df_stefan = pnd.read_csv("/home/knub/Repositories/master-thesis/webapp/out/stefan.txt", sep="\t", header=None)
df_stefan.columns = ["inclusion_idx", "inclusion_id", "method", "words", "intruder", "selected_word", "successful"]
df_stefan

,inclusion_idx,inclusion_id,method,words,intruder,selected_word,successful
0,0,1478882780,lda,"mean,node,graph,algorithm,nodes,tree",mean,mean,True
1,1,1478882769,lda,"energy,optimization,function,analog,solution,problem",analog,analog,True
2,2,1478882934,welda-gaussian,"digits,word,character,phoneme,characters,recognition",phoneme,phoneme,True
3,3,1478882902,welda-gaussian,"source,cells,receptive,visual,motion,direction",source,source,True
4,4,1478882817,lflda,"oscillators,phase,activity,cells,oscillatory,eeg",cells,cells,True
5,5,1478882813,lflda,"vector,distance,transformation,traffic,matching,tangent",traffic,traffic,True
6,6,1478882938,welda-gaussian,"circuits,graph,node,nodes,trees,tree",circuits,circuits,True
7,7,1478882966,welda-gaussian-mixture,"cortical,neuronal,oscillator,oscillators,tree,eeg",tree,tree,True
8,8,1478882901,welda-gaussian,"learning,reinforcement,state,policy,points,action",points,points,True
9,9,1478882866,topicvec,"learned,learning,skills,learn,rates,knowledge",rates,rates,True


In [11]:
for method, df_group in df_stefan.groupby("method"):
    nr_inclusions = len(df_group)
    nr_succ = len(df_group[df_group.successful])
    print "%s: %.2f" % (method, float(nr_succ) / nr_inclusions)

lda: 0.54
lflda: 0.72
topicvec: 0.65
welda-gaussian: 0.74
welda-gaussian-mixture: 0.72
